# 📚 라이브러리 임포트

In [ ]:
import pandas as pd
import re
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter


# 📂 데이터 불러오기 및 전처리

In [ ]:
main_df = pd.read_csv('data/ChatbotData.csv')
main_df = main_df[['Q', 'A']]

def preprocess_text(text):
    text = re.sub(r'[.?!]', '', text)
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

raw_dataframe = main_df.copy()
raw_dataframe['Q_clean'] = raw_dataframe['Q'].apply(preprocess_text)
raw_dataframe['A_clean'] = raw_dataframe['A']
raw_dataframe = raw_dataframe.drop_duplicates(subset='Q_clean')
raw_dataframe = raw_dataframe.drop_duplicates(subset='A_clean')
raw_dataframe = raw_dataframe[raw_dataframe['Q_clean'].apply(len) <= 50]
raw_dataframe = raw_dataframe[raw_dataframe['A_clean'].apply(len) <= 50]

print(f"✅ 전처리 후 문장 수: {len(raw_dataframe)}")


# 🔤 토크나이저 학습

In [ ]:
questions = raw_dataframe['Q_clean'].tolist()
answers = raw_dataframe['A_clean'].tolist()

tokenizer = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(
    questions + answers,
    target_vocab_size=2**13
)

START_TOKEN, END_TOKEN = [tokenizer.vocab_size], [tokenizer.vocab_size + 1]
VOCAB_SIZE = tokenizer.vocab_size + 2
MAX_LENGTH = 25

print(f"✅ Vocab size: {VOCAB_SIZE}")


# 🏗️ 토크나이즈 및 패딩

In [ ]:
def tokenize_and_filter(inputs, outputs):
    tokenized_inputs, tokenized_outputs = [], []
    for (sentence1, sentence2) in zip(inputs, outputs):
        sentence1 = START_TOKEN + tokenizer.encode(sentence1) + END_TOKEN
        sentence2 = START_TOKEN + tokenizer.encode(sentence2) + END_TOKEN
        if len(sentence1) <= MAX_LENGTH and len(sentence2) <= MAX_LENGTH:
            tokenized_inputs.append(sentence1)
            tokenized_outputs.append(sentence2)
    tokenized_inputs = tf.keras.preprocessing.sequence.pad_sequences(
        tokenized_inputs, maxlen=MAX_LENGTH, padding='post')
    tokenized_outputs = tf.keras.preprocessing.sequence.pad_sequences(
        tokenized_outputs, maxlen=MAX_LENGTH, padding='post')
    return tokenized_inputs, tokenized_outputs

questions, answers = tokenize_and_filter(questions, answers)

print(f"✅ 토크나이즈 후 질문 수: {len(questions)}")


# 📚 데이터셋 구성

In [ ]:
BATCH_SIZE = 64
BUFFER_SIZE = 20000

dataset = tf.data.Dataset.from_tensor_slices((questions, answers))
dataset = dataset.cache()
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE)
dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)


# #️⃣ GPT-1 스타일 Decoder-only 모델 구현

In [ ]:
class GPT1Block(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, d_ff, dropout_rate=0.1):
        super(GPT1Block, self).__init__()
        self.attention = tf.keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=d_model)
        self.ffn = tf.keras.Sequential([
            tf.keras.layers.Dense(d_ff, activation='gelu'),
            tf.keras.layers.Dense(d_model)
        ])
        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = tf.keras.layers.Dropout(dropout_rate)
        self.dropout2 = tf.keras.layers.Dropout(dropout_rate)

    def call(self, x, training, mask):
        attn_output = self.attention(x, x, attention_mask=mask)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(x + attn_output)

        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        out2 = self.layernorm2(out1 + ffn_output)
        return out2

class GPT1Model(tf.keras.Model):
    def __init__(self, vocab_size, max_len, d_model=256, num_layers=4, num_heads=8, d_ff=512, dropout_rate=0.1):
        super(GPT1Model, self).__init__()
        self.token_emb = tf.keras.layers.Embedding(vocab_size, d_model)
        self.pos_emb = tf.keras.layers.Embedding(max_len, d_model)
        self.dropout = tf.keras.layers.Dropout(dropout_rate)
        self.blocks = [GPT1Block(d_model, num_heads, d_ff, dropout_rate) for _ in range(num_layers)]
        self.layernorm = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.fc = tf.keras.layers.Dense(vocab_size)

    def call(self, x, training):
        seq_len = tf.shape(x)[1]
        pos = tf.range(start=0, limit=seq_len, delta=1)
        x = self.token_emb(x) + self.pos_emb(pos)
        x = self.dropout(x, training=training)

        mask = tf.linalg.band_part(tf.ones((seq_len, seq_len)), -1, 0)
        mask = mask[tf.newaxis, tf.newaxis, :, :]

        for block in self.blocks:
            x = block(x, training, mask)

        x = self.layernorm(x)
        logits = self.fc(x)
        return logits


# 📈 커스텀 학습률 스케줄러 (CustomSchedule)

In [ ]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, d_model, warmup_steps=4000):
        super(CustomSchedule, self).__init__()
        self.d_model = d_model
        self.d_model = tf.cast(self.d_model, tf.float32)
        self.warmup_steps = warmup_steps

    def __call__(self, step):
        arg1 = tf.math.rsqrt(step)
        arg2 = step * (self.warmup_steps**-1.5)
        return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)


# ⚙️ 모델 컴파일 (손실 함수, 최적화기 설정)

In [ ]:
learning_rate = CustomSchedule(d_model=256)
optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')

def loss_function(y_true, y_pred):
    mask = tf.math.logical_not(tf.math.equal(y_true, 0))
    loss_ = loss_object(y_true, y_pred)
    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask
    return tf.reduce_sum(loss_) / tf.reduce_sum(mask)

model.compile(optimizer=optimizer, loss=loss_function)


# 🛑 EarlyStopping 설정

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='loss',
    patience=3,
    restore_best_weights=True
)


# 🏋️ 모델 학습

In [ ]:
EPOCHS = 50
history = model.fit(dataset, epochs=EPOCHS, callbacks=[early_stopping])


# 🎯 문장 생성 함수

In [ ]:
def generate_sentence(sentence):
    sentence = preprocess_text(sentence)
    input_ids = START_TOKEN + tokenizer.encode(sentence) + END_TOKEN
    input_ids = tf.expand_dims(input_ids, 0)

    for _ in range(MAX_LENGTH):
        predictions = model(input_ids, training=False)
        predictions = predictions[:, -1:, :]
        predicted_id = tf.argmax(predictions, axis=-1)

        if tf.equal(predicted_id[0, 0], END_TOKEN[0]):
            break

        input_ids = tf.concat([input_ids, predicted_id], axis=-1)

    predicted_sentence = tokenizer.decode([i for i in tf.squeeze(input_ids) if i < tokenizer.vocab_size])
    return predicted_sentence


# 🧪 다양한 샘플 테스트 (50개 이상)

In [ ]:
sample_questions = [
    "안녕", "오늘 뭐해?", "날씨 어때?", "배고파", "졸려", "너 뭐하는 애야?", "좋아하는 색은 뭐야?", "심심해", "노래 추천해줘", "영화 볼까?", 
    "운동하고 싶어", "오늘 기분 어때?", "사랑이 뭐야?", "나 지금 우울해", "추천 좀 해줘", "친구랑 싸웠어", "시험 망했어", "맛집 추천해줘", 
    "퇴사하고 싶어", "꿈이 뭐야?", "여행 가고 싶어", "좋아하는 계절은?", "혼자 있는 거 좋아해?", "책 추천해줘", "커피 마실까?", 
    "운동 루틴 알려줘", "오늘 밤 뭐할까?", "외로워", "심심해 죽겠어", "고양이 키울까?", "내일 뭐해?", "지금 어디야?", "잘자", 
    "배달 뭐 시킬까?", "집에 가고 싶어", "게임 추천해줘", "생일 축하해줘", "돈 많이 벌고 싶어", "휴가 가고 싶어", "피곤해 죽겠어", 
    "시간 여행하고 싶어", "오늘 하루 어땠어?", "고백할까 말까?", "힘들어", "사람은 왜 살까?", "세상에서 제일 맛있는 음식은?", 
    "넌 꿈 꿀 수 있어?", "멍 때리고 싶어", "나 우울증 걸린 것 같아", "내 인생은 실패야?", "좋아하는 동물은 뭐야?"
]

for i, q in enumerate(sample_questions, 1):
    print(f"[{i}] ❓ 질문: {q}")
    try:
        response = generate_sentence(q)
        print(f"    ▶️ 답변: {response}")
    except Exception as e:
        print(f"    ⚠️ 오류 발생: {e}")
    print('-' * 60)
